In [1]:
import matplotlib.pyplot as plt
import numpy as np
from formula import Formula
from formula import crossover_formulas
import random
plt.style.use('seaborn-v0_8')

import scikits.bootstrap as bootstrap
import warnings
warnings.filterwarnings('ignore')

import scipy.stats # for finding statistical significance

import copy
import time

In [2]:
QUADRATIC_PATH = './data/quadratic_equation_full_details.csv'
LINEAR_PATH = './data/linear_equations_1_variable.csv'

## Graphing Functions
`plot_mean_and_bootstrapped_ci_over_time`: Plots the mean, bootstrapped 95% C.I over generational time

`plot_mean_over_time`: Plots the means over generational time

In [3]:
def plot_mean_and_bootstrapped_ci_over_time(input_data = None, name = "change me",
                                            x_label = "change me", y_label="change me",
                                            y_limit = None, plot_bootstrap = True):
    """
    
    parameters: 
    input_data: (numpy array of shape (max_k, num_repitions)) solution metric to plot
    name: (string) name for legend
    x_label: (string) x axis label
    y_label: (string) y axis label
    
    returns:
    None
    """
    num_generations = input_data.shape[1]
    num_runs = input_data.shape[0]
    
    # Make plotting the bootstrap optional!
    if plot_bootstrap:
        # confidence interval values need to be calculated separately for each generation

        # So, find the confidence interval over all of the repetitions (:) at each generation,
        # using [:,generation] 
        CIs = np.array([bootstrap.ci(data=input_data[:, generation])
                        for generation in range(num_generations)])
        # Creating np.arrays through list conversion is still a bit cheeky,
        # but it works fine and allows for better indexing (: and ,)

        # Take every generation's (:) lows (0)
        ci_low = CIs[:,0]
        # And take every generation's (:) highs (1)
        ci_high = CIs[:,1]

        plt.fill_between(range(num_generations), ci_low, ci_high, alpha=0.4)
    
    plt.plot(np.mean(input_data, axis=0), label = name)
    
    # Plot annotations
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.ylim(y_limit)
    plt.legend()
    #Save graph to ./graphs
    graph_file_name = f'./graphs/{name}.png'
    plt.savefig(graph_file_name)
    plt.show()


## Evolutionary Algorithms
`basic_evolutionary_algorithm': basic algorithm with truncation selection 

In [4]:
def basic_evolutionary_algorithm(degree = 2, min_len=5, max_len=25,path = None, total_generations=100, num_parents=10, num_children = 10, num_elements_to_mutate=1, num_crossover=2, crossover=True):
    """
    :param degree:  
    :param min_len: 
    :param max_len: 
    :param path: 
    :param total_generations: 
    :param num_parents: 
    :param num_children: 
    :param num_elements_to_mutate: 
    :param num_crossover: 
    :param crossover: 
    :return: 
    """
    #init record keeping
    solution = None
    solution_fitness = -999999
    fitness_over_time = []
    solution_over_time = []
    
    #init population
    Individuals = []
    for _ in range(num_parents):
        Individuals.append(Formula(min_len, max_len, degree, path))
    
    for i in range(total_generations):
        print('Generation: ',i,end='\r')
        
        generation_fitnesses = []
        for ind in Individuals:
            generation_fitnesses.append(ind.eval_fitness())
        fitness_over_time.append(np.max(generation_fitnesses))
        
        children = []
        while len(children) < num_children:
            parent_1, parent_2 = random.sample(Individuals, 2)
            
            # This is greatly faster than deepcopying
            # just make sure to pass the data! File I/O takes forever!
            child_1 = Formula(0, 0, degree, path, data=parent_1.data) #copy.deepcopy(parent_1)
            child_2 = Formula(0, 0, degree, path, data=parent_2.data) #copy.deepcopy(parent_2)
            
            child_1.min_length = parent_1.min_length
            child_1.max_length = parent_1.max_length
            
            child_2.min_length = parent_2.min_length
            child_2.max_length = parent_2.max_length
            
            
#             child_1.fitness = None
#             child_2.fitness = None
            
            #crossover
            if crossover:
                child_1.formula, child_2.formula = crossover_formulas(parent_1, parent_2, num_crossover)
            #mutation
            child_1.mutate_formula(num_elements_to_mutate)
            child_2.mutate_formula(num_elements_to_mutate)
            # add children to population
            children.append(child_1)
            if len(children) < num_children:
                children.append(child_2)

        Individuals.extend(children)
        
        Individuals.sort(key=lambda ind: ind.eval_fitness(), reverse=True)
        Individuals = Individuals[:num_parents]
        #update solutions over time
        solution_over_time.append(Individuals[0].formula)
        if solution is None or Individuals[0].eval_fitness() > solution_fitness:
            solution = Individuals[0]
            solution_fitness = Individuals[0].eval_fitness()
    Individuals[0].pretty_print_formula()
    return fitness_over_time, solution_over_time, solution, solution_fitness
    
    


In [5]:
def tournament_evolutionary_algorithm(degree = 2, min_len=5, max_len=25,path = None, total_generations=100, num_parents=10, num_children = 10, num_elements_to_mutate=1, num_crossover=2, tournament_size = 10, tournament_winners = 2, crossover=True):
    """
    :param degree:  
    :param min_len: 
    :param max_len: 
    :param path: 
    :param total_generations: 
    :param num_parents: 
    :param num_children: 
    :param num_elements_to_mutate: 
    :param num_crossover: 
    :param crossover: 
    :return: 
    """
    #init record keeping
    solution = None
    solution_fitness = -999999
    fitness_over_time = []
    solution_over_time = []
    diversity_over_time = []
    
    #init population
    Individuals = []
    for _ in range(num_parents):
        Individuals.append(Formula(min_len, max_len, degree, path))
        
    for i in range(total_generations):
        print('Generation: ',i,end='\r')
        
        generation_fitnesses = []
        for ind in Individuals:
            generation_fitnesses.append(ind.eval_fitness())
        fitness_over_time.append(np.max(generation_fitnesses))
        # generate children
        children = []
        while len(children) < num_children:
            parent_1, parent_2 = random.sample(Individuals, 2)
            child_1 = Formula(0, 0, degree, path, data=parent_1.data) #copy.deepcopy(parent_1)
            child_2 = Formula(0, 0, degree, path, data=parent_2.data) #copy.deepcopy(parent_2)
            
            child_1.min_length = parent_1.min_length
            child_1.max_length = parent_1.max_length
            
            child_2.min_length = parent_2.min_length
            child_2.max_length = parent_2.max_length
            
            #crossover
            if crossover:
                child_1.formula, child_2.formula = crossover_formulas(parent_1, parent_2, num_crossover)
            #mutation
            child_1.mutate_formula(num_elements_to_mutate)
            child_2.mutate_formula(num_elements_to_mutate)
            # add children to population
            children.append(child_1)
            if len(children) < num_children:
                children.append(child_2)
        
        Individuals.extend(children)
        
        
        # tournament selection
        new_individuals = []
        while len(new_individuals) < num_parents:
            tournament_pool = []
            tournament_pool.extend(random.sample(Individuals, tournament_size))
            tournament_pool.sort(key=lambda ind: ind.eval_fitness(), reverse=True)
            new_individuals.extend(tournament_pool[:tournament_winners])
        # update individuals
        Individuals = new_individuals
        Individuals.sort(key=lambda ind: ind.eval_fitness(), reverse=True)
        #update solutions over time
        solution_over_time.append(Individuals[0].formula)
        if solution is None or Individuals[0].eval_fitness() > solution_fitness:
            solution = Individuals[0]
            solution_fitness = Individuals[0].eval_fitness()
        
    solution.pretty_print_formula()
    return fitness_over_time, solution_over_time, solution, solution_fitness

In [6]:
def age_fitness_pareto_evolution(degree = 2, min_len=5, max_len=25,
                                 path = None, total_generations=100,
                                 num_parents=10, num_children = 10,
                                 num_elements_to_mutate=1, num_crossover=2,
                                 crossover=True, max_fronts = 3,
                                 max_front_prop = 0.50):
    """
    :param degree:  
    :param min_len: 
    :param max_len: 
    :param path: 
    :param total_generations: 
    :param num_parents: 
    :param num_children: 
    :param num_elements_to_mutate: 
    :param num_crossover: 
    :param crossover: 
    :return: 
    """
    #init record keeping
    solution = None
    solution_fitness = -999999
    fitness_over_time = []
    solution_over_time = []
    top_age = []
    
    max_front_individuals = int(max_front_prop * num_parents)
    
    #init population
    Individuals = []
    for _ in range(num_parents):
        Individuals.append(Formula(min_len, max_len, degree, path))
    
    for i in range(total_generations):
        print('Generation: ',i,end='\r')
        
        generation_fitnesses = []
        for ind in Individuals:
            ind.increment_age() # incrememnt age with new generation
            generation_fitnesses.append(ind.eval_fitness())
        fitness_over_time.append(np.max(generation_fitnesses))
        
        children = []
        while len(children) < num_children:
            parent_1, parent_2 = random.sample(Individuals, 2)
            child_1 = Formula(0, 0, degree, path, data=parent_1.data) #copy.deepcopy(parent_1)
            child_2 = Formula(0, 0, degree, path, data=parent_2.data) #copy.deepcopy(parent_2)
            
            child_1.age = parent_1.age
            child_2.age = parent_2.age
            
            child_1.min_length = parent_1.min_length
            child_1.max_length = parent_1.max_length
            
            child_2.min_length = parent_2.min_length
            child_2.max_length = parent_2.max_length
            
            #crossover
            if crossover:
                child_1.formula, child_2.formula = crossover_formulas(parent_1, parent_2, num_crossover)
            #mutation
            child_1.mutate_formula(num_elements_to_mutate)
            child_2.mutate_formula(num_elements_to_mutate)
            # add children to population
            child_1.eval_fitness()
            children.append(child_1)
            
            
            if len(children) < num_children:
                child_2.eval_fitness()
                children.append(child_2)
        Individuals.extend(children)
        #selection procedure (fitness)
        
        
        # START OF PARETO IMPL
        
        # The idea here isn't supposed to be finding the best
        # of age and fitness seperately, but together
        # This is done with the 'pareto front' - which essentially
        # makes a bracket for each value of X, where individuals
        # possessing value X with the the best value Y are chosen
        
        # With the caveat that values superseded in both X AND Y are
        # put into a lower rank
        
        agesorted_individuals = sorted(Individuals, key=lambda individual: individual.get_age())
        
        # The assorted fronts, with the first being the best
        all_fronts = []
        current_age = 0
        
        
        # Take either the first X fronts or the top Y individuals,
        # whichever occurs first (top Y individuals is not a hard bound)
        while len(all_fronts) < max_fronts and sum([len(front) for front in all_fronts]) < max_front_individuals:
            current_age = 0
            current_fitness_best = -99999
            current_front = []
            candidate = None
            candidate_idx = None
            
            old_agesorted_individuals = copy.deepcopy(agesorted_individuals)
            
            for idx, ind in enumerate(old_agesorted_individuals):
                if ind.age > current_age:
                    current_age = ind.age

                    if candidate is not None:
                        current_front.append(candidate)
                        
                        # Make sure not to allow individuals to be double counted!
                        agesorted_individuals.pop(candidate_idx)
                        
                        candidate = None
                        candidate_idx = None

                if ind.eval_fitness() > current_fitness_best:
                    candidate = ind
                    candidate_idx = idx - len(current_front)
                    current_fitness_best = ind.eval_fitness()

            # Make sure to account for the last individual!
            if candidate is not None:
                current_front.append(candidate)
                agesorted_individuals.pop(candidate_idx)
                
            all_fronts.append(current_front)
        
        top_age = current_age
        
        top_individuals = []
        for front in all_fronts:
            top_individuals.extend(front)
        
        Individuals = top_individuals
        
        while len(Individuals) < num_parents:
            Individuals.append(Formula(min_len, max_len, degree, path))
        
        Individuals.sort(key=lambda ind: ind.eval_fitness(), reverse=True)
        
        #update solutions over time
        solution_over_time.append(Individuals[0].formula)
        if solution is None or Individuals[0].eval_fitness() > solution_fitness:
            solution = Individuals[0]
            solution_fitness = Individuals[0].eval_fitness()
    Individuals[0].pretty_print_formula()
    return fitness_over_time, solution_over_time, solution, solution_fitness, top_age
    

In [7]:
experiment_results = {}
solutions_results = {}

In [8]:

num_runs = 20
total_generations = 100
path = './data/quadratic_equation_full_details.csv'
degree = 2
min_len = 5
max_len = 25
num_elements_to_mutate = 1
num_parents = 10
num_children = 10
num_crossover = 2
tournament_size = 10
tournament_winners = 2

# tournament_size = 20
# num_tournament_winners = 10

crossover = True


experiment = 'tournament_2'

experiment_results[experiment] = np.zeros([num_runs, total_generations])
solutions_results[experiment] = {}

for run in range(num_runs):
    fitness, solution, _,_ = tournament_evolutionary_algorithm(degree,
                      
                                                                min_len,
                                                                max_len,
                                                                path,
                                                                total_generations,
                                                                num_parents,
                                                                num_children,
                                                                num_elements_to_mutate,
                                                                num_crossover,
                                                               tournament_size,
                                                               tournament_winners,
                                                                crossover)
    experiment_results[experiment][run] = fitness
    solutions_results[experiment][run] = solution

Generation:  99
 (2-3) 

Generation:  99
 (((5 root 3)-b) root 3) 

Generation:  99
 (((2-c) root 4) root 3) 

Generation:  99
 (2-3) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

 (5/4) 

(34, 'Result too large')

KeyboardInterrupt: 

In [ ]:
num_runs = 20
total_generations = 100
path = './data/linear_equations_1_variable.csv'
degree = 1
min_len = 5
max_len = 10
num_elements_to_mutate = 1
num_parents = 10
num_children = 10
num_crossover = 2

# tournament_size = 20
# num_tournament_winners = 10

crossover = True


experiment = 'basic_linear'

experiment_results[experiment] = np.zeros([num_runs, total_generations])
solutions_results[experiment] = {}

for run in range(num_runs):
    fitness, solution, _,_ = basic_evolutionary_algorithm(degree,
                                                                min_len,
                                                                max_len,
                                                                path,
                                                                total_generations,
                                                                num_parents,
                                                                num_children,
                                                                num_elements_to_mutate,
                                                                num_crossover,
                                                                crossover)
    experiment_results[experiment][run] = fitness
    solutions_results[experiment][run] = solution

In [ ]:
num_runs = 20
total_generations = 100
path = './data/quadratic_equation_full_details.csv'
degree = 2
min_len = 10
max_len = 25
num_elements_to_mutate = 1
num_parents = 10
num_children = 10
num_crossover = 2
# age_select_prop = 0.70

# tournament_size = 20
# num_tournament_winners = 10

crossover = True


experiment = 'pareto_quadratic'

experiment_ages = {}
experiment_ages[experiment] = {}


experiment_results[experiment] = np.zeros([num_runs, total_generations])
solutions_results[experiment] = {}

for run in range(num_runs):
    fitness, solution, _,_, ages = age_fitness_pareto_evolution(degree,
                                                                min_len,
                                                                max_len,
                                                                path,
                                                                total_generations,
                                                                num_parents,
                                                                num_children,
                                                                num_elements_to_mutate,
                                                                num_crossover,
                                                                crossover)
    experiment_ages[experiment][run] = ages
    experiment_results[experiment][run] = fitness
    solutions_results[experiment][run] = solution

In [ ]:
# plot fitness over time
# plotting
plt.figure(1, clear = True)
for experiment in experiment_results.keys():
        plot_mean_and_bootstrapped_ci_over_time(experiment_results[experiment],
                                            experiment,
                                            "Generation",
                                            "Fitness",
                                            [10,20],
                                            plot_bootstrap = False)
plt.show()
timestamp = time.strftime("%Y%m%d_%H%M%S")
filename = f'./graphs/all_data_{timestamp}.png'
plt.savefig(filename)

In [ ]:

# actual quadratic formula:\n",
# (-b (+|-) root((pow(b,2) - 4 * (a * c)),2)) / (2 * a)\n",
q_form = ['a', 'b', 'c', 2, 3, 4, ('*',0,2), ('*',5,6), ('**',1,3), ('-',8,7), ('root',9,3), ('-', 10, 1), ('*',3,0),('/',11,12)]
quad = Formula(min_len, max_len, degree, path)
quad.formula = q_form
quad.length_of_constants = 6
quad.pretty_print_formula()
quad.eval_fitness()



In [ ]:
plot_mean_and_bootstrapped_ci_over_time(input_data=experiment_results['tournament_2'],x_label="Time",y_label='Fitness',
                                        name="Tournament Selection, 10c2")